# Import

In [28]:
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.11.0")
@file:DependsOn("com.fasterxml.jackson.module:jackson-module-kotlin:2.17.1")

In [29]:
// Import necessary libraries
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.variable.PlanningVariable
import ai.timefold.solver.core.api.domain.lookup.PlanningId
import ai.timefold.solver.core.api.domain.solution.PlanningEntityCollectionProperty
import ai.timefold.solver.core.api.domain.solution.ProblemFactCollectionProperty
import ai.timefold.solver.core.api.domain.valuerange.ValueRangeProvider
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore

import java.time.LocalTime
import java.time.DayOfWeek

# Init

In [30]:
// Location class representing a point in Geneva
data class Location(
    val latitude: Double,
    val longitude: Double
)


In [31]:
// Skill enumeration
enum class Skill {
    FIRE_CERTIFICATION,
    BODYGUARD_CERTIFICATION,
    FIRST_AID,
    // Add more skills as needed
}

In [32]:
// Agent class representing a security agent
data class Agent(
    @PlanningId
    val id: Long,
    val name: String,
    val homeLocation: Location,
    val vehicleHomeLocation: Location,
    val skills: Set<Skill>,
    val weeklyWorkloadHours: Int = 40
) {
    // No-arg constructor required for Timefold
    constructor() : this(0, "", Location(0.0, 0.0), Location(0.0, 0.0), emptySet(), 40)
}

In [33]:
// Meeting class representing a client meeting
@PlanningEntity
data class Meeting(
    @PlanningId
    val id: Long,
    val clientName: String,
    val location: Location,
    val requiredSkills: Set<Skill>,
    val durationHours: Int,
    val timeWindowStart: LocalTime,
    val timeWindowEnd: LocalTime,
    val isNightMeeting: Boolean = false
) {
    @PlanningVariable(valueRangeProviderRefs = ["agentRange"], nullable = true)
    var assignedAgent: Agent? = null

    // No-arg constructor required for Timefold
    constructor() : this(0, "", Location(0.0, 0.0), emptySet(), 1, LocalTime.MIN, LocalTime.MAX)
}

# Problem

In [34]:
// Solution class representing the entire problem
@PlanningSolution
data class SecurityAssignment(
    @ProblemFactCollectionProperty
    @ValueRangeProvider(id = "agentRange")
    val agents: List<Agent>,

    @ProblemFactCollectionProperty
    val skills: Set<Skill>,

    @PlanningEntityCollectionProperty
    val meetings: List<Meeting>
) {
    @ai.timefold.solver.core.api.domain.solution.PlanningScore
    var score: HardSoftScore? = null

    // No-arg constructor required for Timefold
    constructor() : this(emptyList(), emptySet(), emptyList())
}

# Fake Data

In [35]:
import kotlin.random.Random

// Function to generate random locations around Geneva
fun generateRandomLocation(): Location {
    // Approximate latitude and longitude for Geneva
    val genevaLat = 46.2044
    val genevaLon = 6.1432
    val latOffset = Random.nextDouble(-0.05, 0.05) // ~5 km
    val lonOffset = Random.nextDouble(-0.05, 0.05)
    return Location(genevaLat + latOffset, genevaLon + lonOffset)
}

// Function to generate fake agents
fun generateAgents(
    numberOfAgents: Int,
    numberOfSkills: Int
): List<Agent> {
    val skills = Skill.values().take(numberOfSkills).toSet()
    return (1..numberOfAgents).map { i ->
        Agent(
            id = i.toLong(),
            name = "Agent $i",
            homeLocation = generateRandomLocation(),
            vehicleHomeLocation = generateRandomLocation(),
            skills = skills.shuffled().take(Random.nextInt(1, skills.size)).toSet(),
            weeklyWorkloadHours = 40
        )
    }
}

// Function to generate fake meetings
fun generateMeetings(
    numberOfMeetings: Int,
    skills: Set<Skill>
): List<Meeting> {
    return (1..numberOfMeetings).map { i ->
        val requiredSkills = skills.shuffled().take(Random.nextInt(1, 3)).toSet()
        val isNight = Random.nextBoolean() && Random.nextDouble() > 0.7 // 30% chance
        val timeWindowStart = if (isNight) LocalTime.of(20, 0) else LocalTime.of(8, 0)
        val timeWindowEnd = if (isNight) LocalTime.of(23, 0) else LocalTime.of(18, 0)
        Meeting(
            id = i.toLong(),
            clientName = "Client $i",
            location = generateRandomLocation(),
            requiredSkills = requiredSkills,
            durationHours = Random.nextInt(1, 3),
            timeWindowStart = timeWindowStart,
            timeWindowEnd = timeWindowEnd,
            isNightMeeting = isNight
        )
    }
}



In [36]:
// Parameters for data generation
val numberOfAgents = 5
val numberOfSkills = 3
val numberOfMeetings = 15

// Generate skills
val allSkills = Skill.values().take(numberOfSkills).toSet()

// Generate agents and meetings
val agents = generateAgents(numberOfAgents, numberOfSkills)
val meetings = generateMeetings(numberOfMeetings, allSkills)

// Create the solution
val problem = SecurityAssignment(
    agents = agents,
    skills = allSkills,
    meetings = meetings
)


# Constraints

In [37]:
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.ConstraintProvider
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore

class SecurityConstraintProvider : ConstraintProvider {
    override fun defineConstraints(constraintFactory: ConstraintFactory): Array<Constraint> {
        return arrayOf(
            // Hard Constraints
            agentSkillConstraint(constraintFactory),
            timeWindowConstraint(constraintFactory),
            weeklyWorkloadConstraint(constraintFactory),
            // Soft Constraints
            minimizeNumberOfAgents(constraintFactory),
            minimizeNightMeetingsOverlap(constraintFactory)
        )
    }

    // Ensure the assigned agent has all required skills
    fun agentSkillConstraint(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Meeting::class.java)
            .filter { meeting ->
                meeting.assignedAgent != null &&
                        !meeting.requiredSkills.all { it in meeting.assignedAgent!!.skills }
            }
            .penalize(HardSoftScore.ONE_HARD)
            .asConstraint("Agent lacks required skills")
    }

    // Ensure the meeting is within the agent's available time window
    fun timeWindowConstraint(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Meeting::class.java)
            .filter { meeting ->
                meeting.assignedAgent != null &&
                        (meeting.timeWindowStart !in agentAvailableHours(meeting.assignedAgent!!) ||
                                meeting.timeWindowEnd !in agentAvailableHours(meeting.assignedAgent!!))
            }
            .penalize(HardSoftScore.ONE_HARD)
            .asConstraint("Meeting outside agent's available time window")
    }

    // Dummy function for agent available hours (to be implemented based on agent's schedule)
    fun agentAvailableHours(agent: Agent): List<LocalTime> {
        // For simplicity, assume agents are available from 8 AM to 6 PM, and 8 PM to 11 PM for night meetings
        val dayHours = (8..18).map { LocalTime.of(it, 0) }
        val nightHours = (20..23).map { LocalTime.of(it, 0) }
        return dayHours + nightHours
    }

    // Ensure agents do not exceed their weekly workload
    fun weeklyWorkloadConstraint(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Agent::class.java)
            .join(constraintFactory.forEach(Meeting::class.java),
                org.optaplanner.core.api.score.stream.Joiners.equal { agent: Agent -> agent },
                org.optaplanner.core.api.score.stream.Joiners.filter { agent, meeting ->
                    meeting.assignedAgent == agent
                }
            )
            .groupBy({ _, meeting -> meeting.assignedAgent },
                ConstraintCollectors.sum(Meeting::durationHours))
            .filter { _, totalHours -> totalHours > 40 }
            .penalize(HardSoftScore.ONE_HARD, { _, excess -> excess })
            .asConstraint("Agent exceeds weekly workload")
    }

    // Soft Constraint: Prefer to use fewer agents
    fun minimizeNumberOfAgents(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Agent::class.java)
            .filter { agent -> agent.meetings.isEmpty() }
            .penalize(HardSoftScore.ONE_SOFT)
            .asConstraint("Minimize number of unused agents")
    }

    // Soft Constraint: Minimize overlapping night meetings for the same agent
    fun minimizeNightMeetingsOverlap(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEachUniquePair(Meeting::class.java,
            org.optaplanner.core.api.score.stream.Joiners.equal(Meeting::assignedAgent),
            org.optaplanner.core.api.score.stream.Joiners.equal { it.isNightMeeting }
        )
            .penalize(HardSoftScore.ONE_SOFT)
            .asConstraint("Minimize overlapping night meetings")
    }
}


Line_35.jupyter.kts (43:29 - 34) Parameter 'agent' is never used
Line_35.jupyter.kts (54:21 - 32) Unresolved reference: optaplanner
Line_35.jupyter.kts (55:21 - 32) Unresolved reference: optaplanner
Line_35.jupyter.kts (55:72 - 77) Cannot infer a type for this parameter. Please specify it explicitly.
Line_35.jupyter.kts (55:79 - 86) Cannot infer a type for this parameter. Please specify it explicitly.
Line_35.jupyter.kts (59:14 - 21) Not enough information to infer type variable ResultContainer_
Line_35.jupyter.kts (60:17 - 37) Unresolved reference: ConstraintCollectors
Line_35.jupyter.kts (61:23 - 24) Cannot infer a type for this parameter. Please specify it explicitly.
Line_35.jupyter.kts (61:26 - 36) Cannot infer a type for this parameter. Please specify it explicitly.
Line_35.jupyter.kts (62:49 - 50) Cannot infer a type for this parameter. Please specify it explicitly.
Line_35.jupyter.kts (62:52 - 58) Cannot infer a type for this parameter. Please specify it explicitly.
Line_35.jup